<a href="https://colab.research.google.com/github/rcadecaro/Alura_Gemini_Botsarello/blob/main/Alura_Botsarello_chatbot_de_atendimento_de_pizzaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Estudo de caso para o Gemini API**:

Agentes e a chamada automática de funções com um chatbot de atendimento de pizzaria.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1hwDVe9cNba5bZlAgBKATsEnoIW6dJpJ5?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Rodar no Google Colab</a>
  </td>
</table>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Botsarello** - Chatbot de atendimento de Pizzaria

## **Agentes de IA** - O assunto do momento

Os agentes de inteligência artificial (IA) estão se tornando cada vez mais importantes em diversos setores devido à sua capacidade de automatizar tarefas, tomar decisões complexas e interagir com o ambiente de forma autônoma.
A execução eficaz de funções é um dos principais fatores que impulsionam a adoção dessa tecnologia, e espera-se que o impacto dos agentes de IA continue crescendo nos próximos anos.

### **Objetivos do projeto**



1.   **Atendimento ao Cliente**: Respondem perguntas, resolvem problemas e fornecem suporte técnico de forma personalizada.
2.   **Tomada de Decisões**: Agentes de IA podem receber algumas funções e tomar a decisão de utilizá-las conforme uma orientação apontada em prompts para atingir um objetivo.



Este notebook mostra um exemplo prático de como usar a chamada automática de funções com o SDK Python da API do Gemini para construir um agente. Iremos definir algumas funções que compõem o sistema de pedidos de pizza, conectá-las à API do Gemini e criar um chat, que será implementado com um loop de interações do agente com o cliente da pizzaria.

**O guia foi inspirado no prompt do Barista bot no estilo ReAct disponível através do AI Studio.**

# **Mãos na massa!**


## Instalação da biblioteca: google-generativeai

In [1]:
!pip install -qU google-generativeai

Para executar este notebook, sua **chave de API** deve estar armazenada na **área protegida do Colab** com o nome **`GOOGLE_API_KEY`**.


In [2]:
from random import randint
from typing import Iterable

import google.generativeai as genai
from google.api_core import retry

from google.colab import userdata
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

## Desenvolvendo uma API simulando o Sistema de Pedidos da Pizzaria

Para simular o sistema de pedidos da sua pizzaria, vamos definir algumas funções para gerenciar o pedido do cliente:.

1. add_to_order - Adicionar
2. remove_item - Editar
3. clear_order - limpar
4. confirm_order - Confirmar
5. place_order - Finalizar

Estas funções rastreiam o pedido do cliente usando as variáveis globais **order** (*o pedido em andamento*) e **placed_order** (*o pedido confirmado enviado para a cozinha*). Cada uma das funções de edição de pedido atualiza o **pedido** e, uma vez confirmado, o pedido é copiado para **pedido_confirmado** e logo em seguida resetado.


In [3]:
order = []  # o pedido em andamento.
placed_order = []  # o pedido confirmado enviado para a cozinha.

def add_to_order(flavor: str, modifiers: Iterable[str] = ()) -> None:
  """Adds the specified flavor to the customer's order, including any modifiers."""
  order.append((flavor, modifiers))


def get_order() -> Iterable[tuple[str, Iterable[str]]]:
  """Returns the customer's order."""
  return order


def remove_item(n: int) -> str:
  """Remove the nth (one-based) item from the order.

  Returns:
    The item that was removed.
  """
  item, modifiers = order.pop(int(n) - 1)
  return item

def send_invoice() -> None:
  """send an invoice to the customer."""
  # TODO(you!): Implement email.
  print('  Nota fical enviada para customer@mail.com')


def clear_order() -> None:
  """Removes all items from the customer's order."""
  order.clear()


def confirm_order() -> str:
  """Asks the customer if the order is correct.

  Returns:
    The user's free-text response.
  """

  print('Seu Pedido:')
  if not order:
    print('  (Nenhum item encontrado)')

  for flavor, modifiers in order:
    print(f'  {flavor}')
    if modifiers:
      print(f'   - {", ".join(modifiers)}')

  return input('O pedido está correto? ')


def place_order() -> int:
  """Submit the order to the kitchen.

  Returns:
    The estimated number of minutes until the order is ready.
  """
  placed_order[:] = order.copy()
  clear_order()
  send_invoice()

  # TODO(you!): Implement pizza fulfilment.
  return randint(1, 10)

## Teste de mesa da API

Vamos testar algumas funções para nos certificarmos que o fluxo de nossa simulação está funcionando como deveria.

In [4]:
# Teste de mesa!

clear_order()
add_to_order('Mussarela', ['borda de catupiry'])
add_to_order('Jiló')
remove_item(2)
add_to_order('Banana', ['Borda de Nutela'])
confirm_order();

Seu Pedido:
  Mussarela
   - borda de catupiry
  Banana
   - Borda de Nutela
O pedido está correto? sim


## Vamos especificar o comportamento do Botsarello no prompt

Aqui vamos vamos definir o prompt do atendente de pizzaria. Nesse prompt vamos passar o cardapio da pizzaria e algumas opções para cada sabor.

As instruções que passaremos devem incluir orientações e exemplos de como as funções devem ser chamadas. (exemplo. "Sempre `confirm_order` com o cliente antes de `place_order`"). Você pode passar o seu proprio estilo de interação para o bot, por exemplo se você quiser que o bot repita o que foi pedido pelo cliente depois de interpretar o pedido, passe instruções para isso.

O final do prompt inclui alguns jargões que o bot pode encontrar, bem como instruções do dia - neste caso, ele observa que a pizzaria está com falta de Jiló.


In [5]:
PIZZA_BOT_PROMPT = """\You are a pizza order taking system and you are restricted to talk only about pizzas on the MENU. Do not talk about anything but ordering MENU pizzas for the customer, ever.
Your goal is to do place_order after understanding the menu items and any modifiers the customer wants.
Add items to the customer's order with add_to_order, remove specific items with remove_item, and reset the order with clear_order.
To see the contents of the order so far, call get_order (by default this is shown to you, not the user)
Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will display the order items to the user and returns their response to seeing the list. Their response may contain modifications.
Always verify and respond with drink and modifier names from the MENU before adding them to the order.
If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect.
You only have the modifiers listed on the menu below: Milk options, espresso shots, caffeine, sweeteners, special requests.
Once the customer has finished ordering items, confirm_order and then place_order.


Horário de Funcionamento: Terças aos Domingos das 5pm às 2am
Preços: Todas as pizzas tem preço fixo de R$ 60,00.

MENU:
Bordas para todas as pizzas:
borda de catupiry
borda de cheddar
borda de parmesão
borda de nutela

Pizzas Classicas:
Mussarela
Napolitana
Calabresa
Marguerita

Pizzas Sobremesa:
Banana
Chocolate
Morango com Chocolate

Pizzas Stranger Things:
Jiló


Modifiers:
borda de catupiry, borda de cheddar, borda de parmesão, borda de nutela
Pedidos Customizados: any reasonable modification that does not involve items not on the menu, for example: 'Sem Queijo', 'Queijo Extra', 'Borda Dupla', etc.


We have run out of Jiló today, so Jiló is not available.
Respond always in Portugues do Brasil.
"""

## Configurando o modelo

Nesta etapa, você reúne as funções em um Ordering_system("sistema") que é passado como `tools`(ferramentas) e instancia o modelo para iniciar a sessão de bate-papo.

Este bloco inclui duas opções para interagir com a API do Gemini. Ao alternar use_sys_inst, você pode alternar entre usar o Gemini 1.5 Pro com uma instrução de sistema (a mais alta qualidade, mas a cota gratuita pode ser insuficiente para uma longa sessão de bate-papo) ou o Gemini 1.0 Pro (cota gratuita mais alta, mas não suporta instruções de sistema).

Uma função send_message retentável também é definida para ajudar em conversas com cota baixa.


### Configurações de criatividade

Vamos usar apenas algumas

| Atributo          | Descrição                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| ----------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `candidate_count` | (Opcional) Número inteiro (`int`) indicando a quantidade de respostas geradas a serem retornadas. Esse valor deve estar entre 1 e 8, inclusive. Se não definido, o padrão será 1.                                                                                                                             |
| `temperature`     | (Opcional) Valor `float` que controla a aleatoriedade da saída. Valores entre 0.0 e 1.0 (inclusive) podem ser usados. Um valor mais próximo de 1.0 produzirá respostas mais variadas e criativas, enquanto um valor mais próximo de 0.0 normalmente resultará em respostas diretas do modelo. |

In [9]:
# Set up the model
# TEMPERATURA / QUANTO MAIOR O MUMERO MAIS IA + CRIATIVA
# TEMPERATURA / QUANTO MENOR O NUMERO MAIS IA + PROFISIONAL/FORMAL

generation_config = {
  "temperature": 0.5,
  "candidate_count": 1,

}

### Configurações de segurança

| Categorias            | Descrições                                                                        |
| --------------------- | --------------------------------------------------------------------------------- |
| Assédio               | Comentários negativos ou nocivos voltados à identidade e/ou atributos protegidos. |
| Discurso de ódio      | Conteúdo grosseiro, desrespeitoso ou linguagem obscena.                           |
| Sexualmente explícito | Contém referências a atos sexuais ou outro conteúdo sexual.                       |
| Material perigoso     | Promove, facilita ou incentiva atos nocivos.                                      |

| Limite (API)                     | Descrição                                                                     |
| -------------------------------- | ----------------------------------------------------------------------------- |
| BLOCK_NONE                       | Sempre mostrar, independentemente da probabilidade de conteúdo não seguro     |
| BLOCK_ONLY_HIGH                  | Bloquear quando houver alta probabilidade de conteúdo não seguro              |
| BLOCK_MEDIUM_AND_ABOVE           | Bloquear quando houver probabilidade média ou alta de conteúdo não seguro     |
| BLOCK_LOW_AND_ABOVE              | Bloquear quando a probabilidade de conteúdo não seguro é baixa, média ou alta |
| HARM_BLOCK_THRESHOLD_UNSPECIFIED | O limite não foi especificado, foi bloqueado usando o limite padrão           |

In [12]:

safety_settings = {
    "HARASSMENT":  "BLOCK_MEDIUM_AND_ABOVE" ,
    "HATE": "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE",
}

In [13]:
ordering_system = [add_to_order, get_order, remove_item, clear_order, confirm_order, place_order]

# Toggle this to switch between Gemini 1.5 with a system instruction, or Gemini 1.0 Pro.
use_sys_inst = False

model_name = 'gemini-1.5-pro-latest' if use_sys_inst else 'gemini-1.0-pro-latest'

if use_sys_inst:
  model = genai.GenerativeModel(
        model_name,
        generation_config=generation_config,
        safety_settings=safety_settings,
        tools=ordering_system,
        system_instruction=PIZZA_BOT_PROMPT
      )
  convo = model.start_chat(enable_automatic_function_calling=True)

else:
  model = genai.GenerativeModel(
        model_name,
        generation_config=generation_config,
        safety_settings=safety_settings,
        tools=ordering_system)

  convo = model.start_chat(
      history=[
          {'role': 'user', 'parts': [PIZZA_BOT_PROMPT, 'Responda apenas em portugues do brasil']},
          {'role': 'model', 'parts': ['Sim eu entendi. Eu darei o meu melhor e irei interagir apenas em portugues do Brasil!']}
        ],
      enable_automatic_function_calling=True)


@retry.Retry(initial=10)
def send_message(message):
  return convo.send_message(message)


placed_order = []
order = []

## Faça seu pedido!

Com o modelo definido e o bate-papo criado, só falta conectar a entrada do usuário ao modelo e exibir a saída, em loop. Esse loop continua até que um pedido completo seja feito.

Quando executado no Colab, qualquer texto de largura fixa se origina do seu código Python (por exemplo, chamadas de print no sistema de pedidos), o texto regular vem da API do Gemini e as caixas delimitadas permitem a entrada do usuário, que é renderizada com um > inicial.

E como já ouvi muitas vezes, **tudo acaba em pizza**!


In [14]:
from IPython.display import display, Markdown

print('Bem vindo a Pizzaria Botsarello! \n')

print("""
Horário de Funcionamento: Terças aos Domingos das 5pm às 2am
Preços: Todas as pizzas tem preço fixo de R$ 60,00.

MENU:
Bordas para todas as pizzas:
borda de catupiry
borda de cheddar
borda de parmesão
borda de nutela


Pizzas Classicas:
Mussarela
Napolitana
Calabresa
Marguerita

Pizzas Sobremesa:
Banana
Chocolate
Morango com Chocolate

Pizzas Stranger Things:
Jiló \n\n""")

while not placed_order:
  response = send_message(input('> '))
  display(Markdown(response.text))


print('\n\n')
print('[Botsarello session over]')
print()
print('Seu Pedido:')
print(f'  {placed_order}\n')
print('- Muito obrigado pela jornada!!')

Bem vindo a Pizzaria Botsarello! 



Horário de Funcionamento: Terças aos Domingos das 5pm às 2am
Preços: Todas as pizzas tem preço fixo de R$ 60,00.

MENU:
Bordas para todas as pizzas:
borda de catupiry
borda de cheddar
borda de parmesão
borda de nutela


Pizzas Classicas:
Mussarela
Napolitana
Calabresa
Marguerita

Pizzas Sobremesa:
Banana
Chocolate
Morango com Chocolate

Pizzas Stranger Things:
Jiló 


> boa tarde, uma pizza de jilo!


Sinto muito, mas não temos mais pizza de Jiló hoje.

> então mussarela com borda de parmessão


Ok, uma pizza de mussarela com borda de parmesão. Mais alguma coisa?

> uma pizza napolitana com borda de catupiry


...

> só isso, pode fechar
Seu Pedido:
  Mussarela
   - borda de parmesão
  Napolitana
   - borda de catupiry
  Banana
   - borda de nutela
O pedido está correto? Uma de cada certo?


Uma de cada certo?

> certo
  Nota fical enviada para customer@mail.com


O seu pedido está confirmado e levará aproximadamente 6 minutos para ficar pronto.




[Botsarello session over]

Seu Pedido:
  [('Mussarela', ['borda de parmesão']), ('Napolitana', ['borda de catupiry']), ('Banana', ['borda de nutela'])]

- Muito obrigado pela jornada!!
